In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
%matplotlib inline 
import matplotlib.pyplot as plt

In [17]:
housing = pd.read_csv('Ames_HousePrice.csv', index_col=0)
real_est = pd.read_csv('Ames Real Estate Data.csv')

In [18]:
housing_c = housing.copy()

In [19]:
housing_c['Baths'] = housing['BsmtFullBath'] + (housing['BsmtHalfBath']/2) + housing['FullBath']+ (housing['HalfBath']/2)

In [20]:
housing_c.drop(columns=['PID','Street','Alley','Utilities','Neighborhood','Condition1',
                    'Condition2','BldgType','HouseStyle', 'RoofMatl',
                    'Exterior1st','Exterior2nd','MasVnrType','BsmtExposure',
                    'BsmtFinType1','BsmtFinType2','Heating','1stFlrSF','2ndFlrSF','PoolQC',
                     'MiscFeature','MiscVal','YrSold','MoSold','SaleType','SaleCondition',
                     'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','MSSubClass',
                     'LotConfig','Foundation','GarageYrBlt'],inplace=True)

## Seperating Quantative and Qualitative data

strings are object datatype so we filter columns by this

In [21]:
quant = [col for col in housing_c.columns if housing_c.dtypes[col] != 'object']
qual = [col for col in housing_c.columns if housing_c.dtypes[col] == 'object']

## Imputing Missing Data (_Quantitative_)

fill missing data with 0s

In [22]:
housing_c[quant].isna().mean().nlargest(8).round(4) * 100

LotFrontage    17.91
MasVnrArea      0.54
Baths           0.08
BsmtFinSF1      0.04
BsmtFinSF2      0.04
BsmtUnfSF       0.04
TotalBsmtSF     0.04
GarageCars      0.04
dtype: float64

In [23]:
housing_c[quant] = housing_c.loc[:, quant].fillna(value=0)

In [24]:
housing_c[quant].isna().mean().nlargest(8).round(4) * 100

GrLivArea       0.0
SalePrice       0.0
LotFrontage     0.0
LotArea         0.0
OverallQual     0.0
OverallCond     0.0
YearBuilt       0.0
YearRemodAdd    0.0
dtype: float64

## Imputing Missing Data (_Qualitative_)

In [25]:
qualdf = housing_c.loc[:, qual]

In [26]:
qualdf.isna().mean().nlargest(17).round(4) * 100

Fence           79.65
FireplaceQu     48.10
GarageFinish     5.00
GarageQual       5.00
GarageCond       5.00
GarageType       4.92
BsmtQual         2.67
BsmtCond         2.67
Electrical       0.04
MSZoning         0.00
LotShape         0.00
LandContour      0.00
LandSlope        0.00
RoofStyle        0.00
ExterQual        0.00
ExterCond        0.00
HeatingQC        0.00
dtype: float64

In [27]:
#kaggle 
for col in qual:
    housing_c[col] = housing_c[col].astype('category')
    if housing_c[col].isnull().any:
        housing_c[col] = housing_c[col].cat.add_categories(['MISSING'])
        housing_c[col] = housing_c[col].fillna('MISSING')

In [28]:
#kaggle
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
        
qual_encoded=[]
for q in qual:
    encode(housing_c, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

['MSZoning_E', 'LotShape_E', 'LandContour_E', 'LandSlope_E', 'RoofStyle_E', 'ExterQual_E', 'ExterCond_E', 'BsmtQual_E', 'BsmtCond_E', 'HeatingQC_E', 'CentralAir_E', 'Electrical_E', 'KitchenQual_E', 'Functional_E', 'FireplaceQu_E', 'GarageType_E', 'GarageFinish_E', 'GarageQual_E', 'GarageCond_E', 'PavedDrive_E', 'Fence_E']


In [29]:
housing_c.drop(qual, inplace=True,axis=1)

In [30]:
housing_c.columns

Index(['GrLivArea', 'SalePrice', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'LowQualFinSF',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'Baths',
       'MSZoning_E', 'LotShape_E', 'LandContour_E', 'LandSlope_E',
       'RoofStyle_E', 'ExterQual_E', 'ExterCond_E', 'BsmtQual_E', 'BsmtCond_E',
       'HeatingQC_E', 'CentralAir_E', 'Electrical_E', 'KitchenQual_E',
       'Functional_E', 'FireplaceQu_E', 'GarageType_E', 'GarageFinish_E',
       'GarageQual_E', 'GarageCond_E', 'PavedDrive_E', 'Fence_E'],
      dtype='object')

In [32]:
housing_c.to_csv('cleanedhousing.csv',index=False)

In [31]:
housing_c.isna().mean().nlargest()

GrLivArea      0.0
SalePrice      0.0
LotFrontage    0.0
LotArea        0.0
OverallQual    0.0
dtype: float64